# Load Balancing Example

In this example we're going to show how you could use various approaches to solve a load balancing problem. Load balancing is when you want to divide a workload as evenly as possible among a set of resources. If we're talking execution times on servers, we can describe it as:

given a list of $n$ execution times, divide them to be executed on $k$ processors so that the total execution time on each processor is as close to the same as possible.

Let's look at a quick example to illustrate this. Say we had 4 jobs that needed to execute and they had the following times: [2,2,4,4]

If we had 2 servers to divide them between, we could have one server do [2,2] and one server do [2,4] or we could have each server do [2,4]. (We could also have one server doing [2,2,4] and one doing [4], but we'll keep it simple for this example.)

Our objective function is total squared deviation of times from balanced times. What does that mean? Well, balanced times is when the total execution time on each server is the same as every other server. But, if our loads are out of balance, some servers will be doing much more work. We want to know the difference between what each server would run if things were in balance and what they are actually running.

Given $s$ is the sum of all execution times, and $k$ is the number of servers, determing our objective function by hand would look like this:

<img src="images/startloadbalancing.png"/>

If we were perfectly load balanced, it would look like this:

<img src="images/endloadbalancing.png"/>

Let's dive into working out this same problem in Python.

First we'll set our times as a numpy array and we'll create an objective function that does all our calculations for us. In order for the objective function to work, we need to know how to assign each execution time. We'll create another numpy array that has the number of the server to assign to, and we'll use index filtering to fetch each "set" of processing times. (We'll walk you through what we're talking about in the comments below.)

In [14]:
import numpy as np
import pandas as pd

#set our execution times
times = np.array([2,4,2,4])
#set our number of servers
k = 2

#set up an array that assigns each execution time to a one of the servers
assign = np.array([0,1,0,1])
#see which times are assigned to server 0
print('These are the times on server 0:', times[assign==0])
print('Total execution time on server 0:', sum(times[assign==0]))
print('These are the times on server 1:', times[assign==1])
print('Total execution time on server 1:', sum(times[assign==1]))


These are the times on server 0: [2 2]
Total execution time on server 0: 4
These are the times on server 1: [4 4]
Total execution time on server 1: 8


### Objective Function
The following balance_metric function is our objective function. It takes in the assignments, the execution times, and the $k$ number of servers. First it calculates the target - the ideal processing time for each server. Then it calculates the squared difference from the target for each server and sums up all the differences to get our total deviation from the ideal.

In [15]:
# objective function = total squared deviation of times from balanced times
def balance_metric(assign,times,k):
    target = sum(times)/k
    return sum( (sum(times[assign==j])-target)**2 for j in range(k) )

Let's determine what our deviation is for our current example.

In [16]:
print('The difference is: ', balance_metric(assign, times, k))

The difference is:  8.0


It's easy to see how our processes should be assigned. So let's fix our assignments to get balanced servers and run our metric again.

In [17]:
#set up an array that assigns each execution time to a one of the servers
assign = np.array([0,0,1,1])
print('The new difference is: ', balance_metric(assign, times, k))

The new difference is:  0.0


Perfect! We now have completely balanced servers.

Of course, we'd typically be dealing with larger loads and possibly more servers and we don't want to balance them by hand. In order to let the algorithms do the work for us, we need a function that reassigns one execution time to a different server. 

Below is the reassign_one function. It takes in the assignments and the number of servers and it swaps an execution time from one server to another.


In [27]:
# define a move function which changes one processor assignment randomly
def reassign_one(assign,k):
    # pick one of the jobs and assign it to one of k processors
    n = len(assign)
    # choose a job and a new processor assignment
    which_job = np.random.randint(0,n,1)[0]
    which_proc = np.random.randint(0,k,1)[0]
    new_assign = assign.copy()
    new_assign[which_job] = which_proc
    return new_assign


Let's walk through some of what this code is doing. First, it's choosing a job to reassign, that's the "which_job" variable. Let's look at what gets assigned to which processor if we just call that line of code with a new, randomly generated set of execution times. We're generating one random integer between 0 and the total number of execution times. This comes back as an array, so we need to fetch just the first item in the array.

In [39]:
#still working with 4 jobs and 2 servers
n = 4
k = 2
#we're going to set some min/max times here for the jobs
min_time = 5
max_time = 10
#randomly generate some jobs
times = np.random.randint(low=min_time, high = max_time, size = n)
#set up a random assignment
assign = np.random.randint(low=0,high=k,size=n)
print('The jobs are:', times)
print('The assignments are:', assign)

#Run the which_job line to see which job we're going to reassign
which_job = np.random.randint(0,n,1)[0]
print('We will reassign job:', which_job)

The jobs are: [8 6 5 5]
The assignments are: [0 0 1 0]
We will reassign job: 1


Then, we can decide which processor to assign it to. We're choose 1 random integer between 0 and the total number of servers. This comes back as an array, so we need to fetch just the first item in the array again.

Then we make a copy of the assignments and use the which_job variable to assign a new processor to a single job execution time.

In [40]:
#this line determines which processor to assign it to:
which_proc = np.random.randint(0,k,1)[0]
print('It will get assigned to server:', which_proc)

#we copy the assignments so that we're not disturbing the original assignments
#then we use the which_job variable to fetch the one we want to reassign and give it the new assignment
new_assign = assign.copy()
new_assign[which_job] = which_proc
print('The new assignments are', new_assign)

It will get assigned to server: 1
The new assignments are [0 1 1 0]


Let's see what happens when we call our function. Call this code a bunch of times and you can see how the assignments change.

In [41]:
reassign_one(assign,k)

array([0, 0, 1, 0])

## Greedy Local Search

Greedy local search is like our hill-climbing examples. We're going to swap one job at a time. If we are closer to being in balance, we'll keep the new assignments. If not, we'll stick with what we had. We'll keep doing this until we hit our maximum no improvement rounds. 

For this algorithm, we don't pass in initial assignments. Our load_balance_local takes care of setting an initial assignment for us.

The function takes in the execution times, the number of servers, and the maximum rounds we'll go without seeing improvement. It returns the best assignment, the smallest deviation from complete balance, and the number of iterations it took to get there.

In [43]:

# local search function
def load_balance_local(times, k, max_no_improve):
    n = len(times)
    # starts from a random assignment to k processors
    current_x = np.random.randint(low=0,high=k,size=n)
    current_f = balance_metric(current_x, times, k)
    best_x = current_x
    best_f = current_f

    # stop search if no better x is found within max_no_improve iterations
    num_moves_no_improve = 0
    iterations = 0
    while (num_moves_no_improve < max_no_improve):
        num_moves_no_improve += 1
        iterations += 1  # just for tracking
        new_x = reassign_one(current_x,k)
        new_f = balance_metric(new_x, times, k)
        if new_f < current_f:
            num_moves_no_improve = 0
            current_x = new_x
            current_f = new_f
            if current_f < best_f:  
                best_x = current_x  
                best_f = current_f
    return best_x, best_f, iterations

Let's run this with a small number of servers and a small number of execution times. Does your deviation hit zero?

In [64]:
# generate random job times
np.random.seed(666) #comment this out to play with new numbers
#we'll start with 100 execution times
n = 20
#we'll start with 5 servers
k = 2
min_time = 20
max_time = 200
times = np.random.randint(low=min_time, high = max_time, size = n)

In [70]:
best_assign, best_f, num_iter = load_balance_local(times,k,5000)
print('The best assignment is', best_assign)
print('The deviation from balance is', best_f)
print('It took', num_iter, 'iterations.')

The best assignment is [0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1]
The deviation from balance is 544.5
It took 5029 iterations.


### Checking our work
We can look at the total time on each processor to see how close they were. Ideally, they'd be the same. 

In [72]:
# total time on each processor ... should be about the same
[ sum(times[best_assign==j]) for j in range(k)]

[1204, 1171]

There's randomness here, so some runs will be better than others. But, even if it seems like your best deviation wasn't that great, it was probably still better than random assignments. Let's compare by doing a random assignment of the processes and see what the time was on each server.

In [73]:
# for comparison here are total times on each processor for random assignment
assign = np.random.choice(k, size = n, replace = True)
[ sum(times[assign==j]) for j in range(k)]

[1641, 734]